In [202]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
#import time
import csv
import requests

session = requests.Session()
hdr = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.121 Safari/537.36',
           'Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
           'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
           'Accept-Encoding': 'none',
           'Accept-Language': 'en-US,en;q=0.8',
           'Connection': 'keep-alive'}

In [203]:
url = 'https://www.visitgainesville.com/events/list/?tribe_redirected=true'
req = session.get(url, headers=hdr)
bsObj = BeautifulSoup(req.text, "html.parser")

In [204]:
# create empty list to hold calender content
calenderList = []

def get_event():
    List = bsObj.find_all('div', {'class':'type-tribe_events'})
    
    for event in List:
    
        Event_Name = event.find('h2', {'class':'tribe-events-list-event-title'}).get_text().strip('\r\n\t ')
        Link = event.a['href']
        Date_start = event.find('span', {'class':'tribe-event-date-start'}).get_text()
        
        try:
            Date_end = event.find('span', {'class':'tribe-event-date-end'}).get_text()
        except:
            Date_end = "none"
        try:        
            Address = event.find('span', {'class':'tribe-street-address'}).get_text().strip('\r\n\t ').replace('\n','')
        except:
            Address = "n.a."
            
        City = event.find('span', {'class':'tribe-locality'}).get_text().strip('\r\n\t ').replace('\n','')
        
        eventList = [Event_Name, Date_start, Date_end, Address, City, Link]
        calenderList.append(eventList)
   
    #return event
    #I took this out function cal to function(get_event()), because it was printing twice with the loop below... beacuse I started with range 1.
#get_event()    

    

In [205]:
for count in range(1, 10):
    url = ('https://www.visitgainesville.com/events/list/?tribe_event_display=list&tribe_paged=')
    new_url = (url + str(count))
    req = session.get(new_url, headers=hdr)
    bsObj = BeautifulSoup(req.text, "html.parser")
    get_event()    

In [206]:
#WRITE SCRAPER INTO CSV FILE

def write_csv(calenderList):
    # open new file for writing - named it calender.csv 
    newfile = open('gainesville_Calender.csv', 'w', encoding='utf-8')

    # make a Python CSV read? object -
    c = csv.writer(newfile)
    
    # write the column headings row 
    c.writerow(['Event_Name', 'Date_start', 'Date_end', 'Address', 'City', 'Link'])
    
    # Write calenderList into CSV, you don't need a loop because it has already looped above!
    for item in calenderList:
        c.writerow(item)
    # close the file - end of function 
    newfile.close()
    
    # function does not need a return - but the next line prevents an error in Jupyter, so keep it 
    return None

# run the function - this should create a .csv file 
write_csv(calenderList)

In [ ]:
#https://gainesvilledowntown.com/downtown-calendar/
#div class="greatwp-main-wrapper